#  Dataset: House Prices - Advanced Regression Techniques

---

##  Objetivo del dataset
- **Meta principal**: **Predecir el precio de venta de una casa (`SalePrice`)** en la ciudad de Ames (Iowa, EE.UU.).
- **Cómo se logra**: mediante modelos de **regresión múltiple** u otros algoritmos de Machine Learning, usando como insumo las características de la casa (tamaño, materiales, condiciones, ubicación, etc.).

---

##  Pasos para llegar al objetivo
1. **Comprensión del problema**  
   - Variable dependiente: `SalePrice`.
   - Variables independientes: características estructurales, de ubicación, materiales y extras.

2. **Exploración de los datos (EDA)**  
   - Revisar registros, valores nulos, tipos de datos.
   - Generar estadísticas descriptivas y distribuciones.

3. **Limpieza de datos**  
   - Imputar o eliminar valores faltantes (`NA`).
   - Tratar valores atípicos.
   - Agrupar categorías poco frecuentes.

4. **Codificación de variables categóricas**  
   - Usar *One Hot Encoding* o *Label Encoding*.

5. **Escalamiento / Normalización**  
   - Aplicar sobre variables numéricas cuando el modelo lo requiera.

6. **Selección de variables relevantes**  
   - Usar correlaciones, ANOVA o técnicas de *feature selection*.

7. **Construcción del modelo**  
   - Empezar con **Regresión Lineal Múltiple**.
  

8. **Evaluación del modelo**  
- **RMSE (Root Mean Squared Error):** mide el error promedio, penaliza más los errores grandes.  
- **MAE (Mean Absolute Error):** error promedio absoluto entre lo real y lo predicho.  
- **R² (Coeficiente de determinación):** indica qué tan bien el modelo explica la variabilidad (0 a 1).  
- **Validación cruzada:** divide los datos en partes para entrenar y probar el modelo de forma más robusta.  


9. **Interpretación de resultados**  
   - Identificar qué variables influyen más en el precio de venta.

---

##  Resumen de variables principales
El dataset contiene **79 variables**. Algunos ejemplos clave:

- **Lote**
  - `MSSubClass`: tipo de vivienda (1 piso, 2 pisos, dúplex, PUD).  
  - `MSZoning`: zonificación (residencial, comercial, agrícola).  
  - `LotFrontage`: longitud de fachada.  
  - `LotArea`: área del terreno.  
  - `LotShape`: forma del lote (regular, irregular).  
  - `LandContour`: topografía (plano, pendiente, depresión).  
  - `Neighborhood`: barrio donde se ubica la casa.  

- **Construcción**
  - `BldgType`: tipo de construcción (unifamiliar, dúplex, townhouse).  
  - `HouseStyle`: estilo (1 piso, 2 pisos, split-level).  
  - `OverallQual`: calidad general (1 = muy mala, 10 = excelente).  
  - `OverallCond`: estado general de la vivienda.  
  - `YearBuilt`: año de construcción.  
  - `YearRemodAdd`: año de remodelación.  
  - `RoofStyle` / `RoofMatl`: tipo y material del techo.  
  - `Exterior1st` / `Exterior2nd`: material exterior.  

- **Sótano**
  - `BsmtQual`, `BsmtCond`: calidad y estado del sótano.  
  - `BsmtExposure`: exposición (ventanas, walkout).  
  - `TotalBsmtSF`: área total del sótano.  

- **Habitabilidad**
  - `GrLivArea`: área habitable sobre el nivel del suelo.  
  - `FullBath`, `HalfBath`: baños.  
  - `Bedroom`, `Kitchen`: habitaciones y cocinas.  
  - `TotRmsAbvGrd`: total de cuartos.  

- **Extras**
  - `Fireplaces`, `FireplaceQu`: chimeneas.  
  - `GarageType`, `GarageCars`, `GarageArea`: características del garaje.  
  - `WoodDeckSF`, `OpenPorchSF`, `PoolArea`, `Fence`: áreas exteriores y amenidades.  

- **Venta**
  - `MoSold`, `YrSold`: mes y año de venta.  
  - `SaleType`: tipo de venta (convencional, cash, VA loan).  
  - `SaleCondition`: condiciones de la venta (normal, familiar, parcial, etc.).  

- **Target**
  - `SalePrice`: precio de venta de la vivienda (variable a predecir).

---

##  En resumen
El dataset busca **explicar y predecir el precio de las casas** usando información estructural, de ubicación, calidad de materiales, extras y condiciones de venta.


In [ ]:


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import os


print(" Carpeta actual:", os.getcwd())
print(" Archivos disponibles:", os.listdir())

if not os.path.exists("train.csv"):
    raise FileNotFoundError("No se encontró 'train.csv'")
if not os.path.exists("test.csv"):
    raise FileNotFoundError("No se encontró 'test.csv'")


train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print("Archivos cargados correctamente")
print(f"Tamaño train: {train.shape}")
print(f"Tamaño test: {test.shape}")


# Eliminar columnas con más del 40% de valores nulos
missing_ratio = train.isnull().mean()
cols_to_drop = missing_ratio[missing_ratio > 0.4].index
train = train.drop(columns=cols_to_drop)
test = test.drop(columns=cols_to_drop, errors='ignore')

# Imputar valores faltantes numéricos (mediana)
for col in train.select_dtypes(include=[np.number]).columns:
    train[col] = train[col].fillna(train[col].median())
    if col in test.columns:
        test[col] = test[col].fillna(test[col].median())

# Imputar valores faltantes categóricos (moda)
for col in train.select_dtypes(exclude=[np.number]).columns:
    train[col] = train[col].fillna(train[col].mode()[0])
    if col in test.columns:
        test[col] = test[col].fillna(test[col].mode()[0])


train_encoded = pd.get_dummies(train, drop_first=True)
test_encoded = pd.get_dummies(test, drop_first=True)


if "SalePrice" in test_encoded.columns:
    test_encoded = test_encoded.drop("SalePrice", axis=1)

# Alinear columnas (sin incluir la etiqueta)
X = train_encoded.drop("SalePrice", axis=1)
y = train_encoded["SalePrice"]
X, test_encoded = X.align(test_encoded, join='left', axis=1, fill_value=0)


X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
test_scaled = scaler.transform(test_encoded)

model = RandomForestRegressor(n_estimators=300, random_state=42)
model.fit(X_train_scaled, y_train)


predictions = model.predict(test_scaled)

if "Id" not in test.columns:
    test["Id"] = np.arange(1, len(test) + 1)

pred_df = pd.DataFrame({
    "Id": test["Id"],
    "SalePrice": predictions
})

pred_df.to_csv("predicciones.csv", index=False)
print(" Archivo 'predicciones.csv' generado con éxito.")


📁 Carpeta actual: c:\Users\Fernanda\Downloads\ELECTIVA-PROFESIONAL-III-main (4)\ELECTIVA-PROFESIONAL-III-main\Semana 6\house-prices-advanced-regression-techniques
📄 Archivos disponibles: ['ayuda.py', 'data_description.txt', 'predicciones.csv', 'sample_submission.csv', 'taller.ipynb', 'test.csv', 'train.csv']
✅ Archivos cargados correctamente
Tamaño train: (1460, 81)
Tamaño test: (1459, 80)
✅ Archivo 'predicciones.csv' generado con éxito.
